# Рад са људима у петљи уз Microsoft Agent Framework

## 🎯 Циљеви учења

У овом нотебуку ћете научити како да имплементирате **рад са људима у петљи** користећи `RequestInfoExecutor` из Microsoft Agent Framework-а. Овај моћан образац омогућава паузирање AI токова рада ради прикупљања људских уноса, чинећи ваше агенте интерактивним и дајући људима контролу над критичним одлукама.

## 🔄 Шта је рад са људима у петљи?

**Рад са људима у петљи (HITL)** је образац дизајна где AI агенти паузирају извршење како би затражили људски унос пре наставка. Ово је важно за:

- ✅ **Критичне одлуке** - Добијање људског одобрења пре предузимања важних акција
- ✅ **Нејасне ситуације** - Омогућавање људима да разјасне када AI није сигуран
- ✅ **Корисничке преференције** - Питање корисника да изаберу између више опција
- ✅ **Усклађеност и безбедност** - Осигурање људског надзора за регулисане операције
- ✅ **Интерактивна искуства** - Изградња конверзацијских агената који реагују на кориснички унос

## 🏗️ Како функционише у Microsoft Agent Framework-у

Оквир пружа три кључне компоненте за HITL:

1. **`RequestInfoExecutor`** - Посебан извршилац који паузира ток рада и емитује `RequestInfoEvent`
2. **`RequestInfoMessage`** - Основна класа за типизиране захтеве који се шаљу људима
3. **`RequestResponse`** - Повезује људске одговоре са оригиналним захтевима користећи `request_id`

**Образац тока рада:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Наш пример: Резервација хотела уз потврду корисника

Проширићемо условни ток рада додавањем људске потврде **пре** предлагања алтернативних дестинација:

1. Корисник захтева дестинацију (нпр. "Париз")
2. `availability_agent` проверава да ли су собе доступне
3. **Ако нема соба** → `confirmation_agent` пита "Да ли желите да видите алтернативе?"
4. Ток рада **паузира** користећи `RequestInfoExecutor`
5. **Људски одговор** "да" или "не" преко уноса у конзоли
6. `decision_manager` усмерава на основу одговора:
   - **Да** → Приказ алтернативних дестинација
   - **Не** → Отказивање захтева за резервацију
7. Приказ коначног резултата

Ово показује како да корисницима дате контролу над предлозима агента!

---

Хајде да почнемо! 🚀


## Корак 1: Увоз потребних библиотека

Увозимо стандардне компоненте оквира за агенте, као и **класе специфичне за људску интеракцију**:
- `RequestInfoExecutor` - Извршилац који паузира ток рада ради уноса од стране човека
- `RequestInfoEvent` - Догађај који се емитује када се захтева унос од стране човека
- `RequestInfoMessage` - Основна класа за типизиране захтеве
- `RequestResponse` - Повезује одговоре људи са захтевима
- `WorkflowOutputEvent` - Догађај за откривање излаза токова рада


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Корак 2: Дефинишите Pydantic моделе за структурисане излазе

Ови модели дефинишу **шему** коју ће агенти враћати. Задржавамо све моделе из условног радног процеса и додајемо:

**Ново за људску интервенцију:**
- `HumanFeedbackRequest` - Подкласа `RequestInfoMessage` која дефинише податке захтева који се шаљу људима
  - Садржи `prompt` (питање које треба поставити) и `destination` (контекст о недоступном граду)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Корак 3: Направите алат за резервацију хотела

Исти алат из условног радног процеса - проверава да ли су собе доступне на одредишту.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Корак 4: Дефинисање функција услова за рутирање

Потребне су нам **четири функције услова** за наш радни ток са људском интервенцијом:

**Из условног радног тока:**
1. `has_availability_condition` - Рутира када су хотели ДОСТУПНИ
2. `no_availability_condition` - Рутира када хотели НИСУ ДОСТУПНИ

**Ново за радни ток са људском интервенцијом:**
3. `user_wants_alternatives_condition` - Рутира када корисник каже "да" за алтернативе
4. `user_declines_alternatives_condition` - Рутира када корисник каже "не" за алтернативе


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Корак 5: Креирање извршиоца Decision Manager-а

Ово је **срж обрасца са људским учешћем**! `DecisionManager` је прилагођени `Executor` који:

1. **Прима повратне информације од људи** преко `RequestResponse` објеката
2. **Обрађује корисникову одлуку** (да/не)
3. **Управља током рада** слањем порука одговарајућим агентима

Кључне карактеристике:
- Користи декоратор `@handler` за излагање метода као корака у току рада
- Прима `RequestResponse[HumanFeedbackRequest, str]` који садржи и оригинални захтев и корисников одговор
- Генерише једноставне поруке "да" или "не" које активирају наше функције услова


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Корак 6: Креирајте прилагођени извршилац приказа

Исти извршилац приказа из условног тока рада - даје коначне резултате као излаз тока рада.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Корак 7: Учитајте променљиве окружења

Конфигуришите LLM клијента (GitHub Models, Azure OpenAI, или OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Корак 8: Креирање AI агената и извршилаца

Креирамо **шест компоненти радног процеса**:

**Агенти (упаковани у AgentExecutor):**
1. **availability_agent** - Проверава доступност хотела користећи алат
2. **confirmation_agent** - 🆕 Припрема захтев за потврду од стране корисника
3. **alternative_agent** - Предлаже алтернативне градове (када корисник каже да)
4. **booking_agent** - Подстиче резервацију (када су собе доступне)
5. **cancellation_agent** - 🆕 Обрађује поруку о отказивању (када корисник каже не)

**Посебни извршиоци:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` који паузира радни процес ради уноса од стране корисника
7. **decision_manager** - 🆕 Прилагођени извршилац који усмерава на основу одговора корисника (већ дефинисан горе)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Корак 9: Изградња радног процеса са људском интервенцијом

Сада конструишемо графикон радног процеса са **условним рутирањем**, укључујући пут са људском интервенцијом:

**Структура радног процеса:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Кључне везе:**
- `availability_agent → confirmation_agent` (када нема слободних соба)
- `confirmation_agent → prepare_human_request` (промена типа)
- `prepare_human_request → request_info_executor` (пауза за људску интервенцију)
- `request_info_executor → decision_manager` (увек - пружа RequestResponse)
- `decision_manager → alternative_agent` (када корисник каже "да")
- `decision_manager → cancellation_agent` (када корисник каже "не")
- `availability_agent → booking_agent` (када има слободних соба)
- Сви путеви се завршавају на `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Корак 10: Покрените тест случај 1 - Град БЕЗ доступности (Париз са људском потврдом)

Овај тест демонстрира **потпуни циклус са људским учешћем**:

1. Захтев за хотел у Паризу
2. availability_agent проверава → Нема слободних соба
3. confirmation_agent креира питање за људску потврду
4. request_info_executor **паузира ток рада** и емитује `RequestInfoEvent`
5. **Апликација детектује догађај и упућује корисника у конзоли**
6. Корисник уноси "да" или "не"
7. Апликација шаље одговор преко `send_responses_streaming()`
8. decision_manager усмерава на основу одговора
9. Приказује се коначни резултат

**Кључни образац:**
- Користите `workflow.run_stream()` за прву итерацију
- Користите `workflow.send_responses_streaming(pending_responses)` за наредне итерације
- Слушајте `RequestInfoEvent` да бисте детектовали када је потребан људски унос
- Слушајте `WorkflowOutputEvent` да бисте ухватили коначне резултате


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Корак 11: Покрените тест случај 2 - Град СА доступношћу (Стокхолм - Није потребан људски унос)

Овај тест демонстрира **директан пут** када су собе доступне:

1. Затражите хотел у Стокхолму  
2. availability_agent проверава → Собе доступне ✅  
3. booking_agent предлаже резервацију  
4. display_result приказује потврду  
5. **Људски унос није потребан!**

Радни ток у потпуности заобилази пут са људским учешћем када су собе доступне.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Кључне информације и најбоље праксе за људску интервенцију

### ✅ Шта сте научили:

#### 1. **RequestInfoExecutor образац**
Образац за људску интервенцију у Microsoft Agent Framework-у користи три кључне компоненте:
- `RequestInfoExecutor` - Паузира ток рада и емитује догађаје
- `RequestInfoMessage` - Основна класа за типизиране захтеве (наследите ову класу!)
- `RequestResponse` - Повезује људске одговоре са оригиналним захтевима

**Критично разумевање:**
- `RequestInfoExecutor` НЕ прикупља унос сам - само паузира ток рада
- Ваш апликативни код мора слушати `RequestInfoEvent` и прикупљати унос
- Морате позвати `send_responses_streaming()` са речником који мапира `request_id` на кориснички одговор

#### 2. **Образац за стриминг извршења**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Архитектура заснована на догађајима**
Слушајте специфичне догађаје за контролу тока рада:
- `RequestInfoEvent` - Потребан је људски унос (ток рада паузиран)
- `WorkflowOutputEvent` - Коначни резултат је доступан (ток рада завршен)
- `WorkflowStatusEvent` - Промене стања (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, итд.)

#### 4. **Прилагођени извршиоци са @handler**
`DecisionManager` показује како креирати извршиоце који:
- Користе декоратор `@handler` за излагање метода као корака у току рада
- Примају типизиране поруке (нпр. `RequestResponse[HumanFeedbackRequest, str]`)
- Усмеравају ток рада слањем порука другим извршиоцима
- Приступају контексту преко `WorkflowContext`

#### 5. **Условно усмеравање са људским одлукама**
Можете креирати функције услова које процењују људске одговоре:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Примена у стварном свету:

1. **Токови одобрења**
   - Добијање одобрења менаџера пре обраде извештаја о трошковима
   - Захтевање људске ревизије пре слања аутоматизованих имејлова
   - Потврда трансакција велике вредности пре извршења

2. **Модерација садржаја**
   - Означавање сумњивог садржаја за људску ревизију
   - Тражење од модератора да донесу коначну одлуку у граничним случајевима
   - Ескалација ка људима када је самопоуздање АИ ниско

3. **Корисничка подршка**
   - Омогућавање АИ да аутоматски обрађује рутинска питања
   - Ескалација сложених проблема ка људским агентима
   - Питање корисника да ли жели да разговара са човеком

4. **Обрада података**
   - Тражење од људи да реше нејасне уносе података
   - Потврда АИ интерпретација нејасних докумената
   - Омогућавање корисницима да бирају између више ваљаних интерпретација

5. **Системи критични за безбедност**
   - Захтевање људске потврде пре неповратних радњи
   - Добијање одобрења пре приступа осетљивим подацима
   - Потврда одлука у регулисаним индустријама (здравство, финансије)

6. **Интерактивни агенти**
   - Изградња конверзацијских ботова који постављају додатна питања
   - Креирање чаробњака који воде кориснике кроз сложене процесе
   - Дизајнирање агената који сарађују са људима корак по корак

### 🔄 Поређење: Са и без људске интервенције

| Карактеристика | Условни ток рада | Ток рада са људском интервенцијом |
|----------------|------------------|----------------------------------|
| **Извршење** | Један `workflow.run()` | Петља са `run_stream()` + `send_responses_streaming()` |
| **Кориснички унос** | Нема (потпуно аутоматизовано) | Интерактивни упити преко `input()` или UI |
| **Компоненте** | Агенти + Извршиоци | + RequestInfoExecutor + DecisionManager |
| **Догађаји** | Само AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, итд. |
| **Паузирање** | Нема паузирања | Ток рада се паузира на RequestInfoExecutor |
| **Контрола од стране људи** | Нема контроле од стране људи | Људи доносе кључне одлуке |
| **Примена** | Аутоматизација | Сарадња и надзор |

### 🚀 Напредни обрасци:

#### Више тачака за људске одлуке
Можете имати више `RequestInfoExecutor` чворова у истом току рада:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Руковање временским ограничењима
Примените временска ограничења за људске одговоре:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Интеграција са напредним корисничким интерфејсом
Уместо `input()`, интегришите са веб интерфејсом, Slack-ом, Teams-ом, итд.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Условна људска интервенција
Тражите људски унос само у специфичним ситуацијама:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Најбоље праксе:

1. **Увек наследите RequestInfoMessage**
   - Омогућава типску сигурност и валидацију
   - Обезбеђује богат контекст за приказ у корисничком интерфејсу
   - Јасно дефинише намеру сваког типа захтева

2. **Користите описне упите**
   - Укључите контекст о томе шта питате
   - Објасните последице сваког избора
   - Држите питања једноставним и јасним

3. **Руковање неочекиваним уносом**
   - Валидација корисничких одговора
   - Обезбеђивање подразумеваних вредности за неважећи унос
   - Давање јасних порука о грешци

4. **Праћење ID-ева захтева**
   - Користите корелацију између request_id и одговора
   - Не покушавајте ручно управљати стањем

5. **Дизајн за неблокирање**
   - Не блокирајте нити чекајући унос
   - Користите асинхроне обрасце
   - Подржите истовремене инстанце токова рада

### 📚 Повезани концепти:

- **Agent Middleware** - Пресретање позива агента (претходни бележник)
- **Управљање стањем токова рада** - Перзистенција стања токова рада између покретања
- **Сарадња више агената** - Комбинација људске интервенције са тимовима агената
- **Архитектуре засноване на догађајима** - Изградња реактивних система са догађајима

---

### 🎓 Честитамо!

Овладали сте токовима рада са људском интервенцијом у Microsoft Agent Framework-у! Сада знате како да:
- ✅ Паузирате токове рада ради прикупљања људског уноса
- ✅ Користите RequestInfoExecutor и RequestInfoMessage
- ✅ Руковање стриминг извршењем са догађајима
- ✅ Креирате прилагођене извршиоце са @handler
- ✅ Усмеравате токове рада на основу људских одлука
- ✅ Изградите интерактивне АИ агенте који сарађују са људима

**Ово је критичан образац за изградњу поузданих и контролисаних АИ система!** 🚀



---

**Одрицање од одговорности**:  
Овај документ је преведен помоћу услуге за превођење уз помоћ вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
